In [53]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from tbparse import SummaryReader


models = {
    "NeRF (sig=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "NeRF (sig=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "NeRF (sig=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    "NeRF (sig=200)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_200/ckpt/cf.txt",
    # "NeRF (mean 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mean_0.1/ckpt/cf.txt",
    # "NeRF (mean 0.9)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mean_0.9/ckpt/cf.txt",
    # "NeRF (mode 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mode_0.1/ckpt/cf.txt",
    # "NeRF (mode 0.9)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mode_0.9/ckpt/cf.txt",
    "NeRF (test mean 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mean_0.1/ckpt/cf.txt",
    "NeRF (test mode 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/ckpt/cf.txt",

    # "ours (full_init_log_raw_a_sparse_58)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/full_init_log_raw_a_sparse/0058/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_log_raw_a_sparse_58_mode_09)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_log_raw_a_sparse_0058/ckpt_eval_surf_trav/depth_mode_0.9/ckpt/cf.txt",
    # "ours (full_init_log_raw_a_sparse_63)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/full_init_log_raw_a_sparse/0063/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_3)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/full_init_l2_less_reg/0003/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_7)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/full_init_l2_less_reg/0007/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_25)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/full_init_l2_less_reg/0025/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (norm_match_both_1)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/norm_match_both/0001/ckpt_eval_surf_trav/ckpt/cf.txt",

#     "ours (slow_surf_mlv_65/mode)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/slow_surf_mlv/0065/ckpt_eval_depth_mode_0.1/ckpt/cf.txt",
#     "ours (slow_surf_mlv_65/nc)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/slow_surf_mlv/0065/ckpt_eval_surf_trav/ckpt/cf.txt",
#     "ours (slow_surf_mlv_71/mode)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/slow_surf_mlv/0071/ckpt_eval_depth_mode_0.1/ckpt/cf.txt",
#     "ours (slow_surf_mlv_71/nc)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/slow_surf_mlv/0071/ckpt_eval_surf_trav/ckpt/cf.txt",
#     "ours (slow_surf_mlv_2_80/mode)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/slow_surf_mlv_2/0080/ckpt_eval_depth_mode_0.1/ckpt/cf.txt",
#     "Ours (mlv2_16_slow_alpha_4/mode)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/slow_mlv2_fix/16_slow_alpha_4/ckpt_eval_depth_mode_0.1/ckpt/cf.txt",
#     "Ours (mlv2_16_slow_alpha_4/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/slow_mlv2_fix/16_slow_alpha_4/ckpt_eval_all_surf/ckpt/cf.txt",
    
    # "Ours (mlv_trunc_tv/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/mlv_trunc_tv/0005/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "Ours (mlv_trunc_tv/mode)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/mlv_trunc_tv/0005/ckpt_eval_depth_mode_0.1/ckpt/cf.txt",
    
    "Ours (mlv_trunc_conv_2_28/surf_closed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/mlv_trunc_conv_2/0028/ckpt_eval_surf_trav_surf_close/ckpt/cf.txt",

    "Ours (lv_nrm_mlv_50/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/lv_nrm_mlv/0050/ckpt_eval_surf_trav/ckpt/cf.txt",
 
    # "ours (full_init_l2_less_reg_0003)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mode01)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003/ckpt_eval_surf_trav/depth_mode_0.1/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mode09)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003/ckpt_eval_surf_trav/depth_mode_0.9/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mlv)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003_mlv/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mlv_mode01)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003_mlv/ckpt_eval_surf_trav/depth_mode_0.1/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mlv_mode09)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003_mlv/ckpt_eval_surf_trav/depth_mode_0.9/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mlv_2)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003_mlv_2/ckpt_eval_surf_trav/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mlv_2_mode01)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003_mlv_2/ckpt_eval_surf_trav/depth_mode_0.1/ckpt/cf.txt",
    # "ours (full_init_l2_less_reg_0003_mlv_2_mode09)" : "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good/full_init_l2_less_reg_0003_mlv_2/ckpt_eval_surf_trav/depth_mode_0.9/ckpt/cf.txt",

    "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exp/{}/womask_hfs/fine_mesh/cf.txt",
}

def read_entry(p):
    entry = {
        # "Mean d2s": None,
        # "Mean s2d": None,
        "Avg cf": None,
        # "Accuracy": None,
        # "Completeness": None,
        "F1": None,
        "psnr": None,
    }
    if Path(p).exists():
        with Path(p).open('r') as f:
            lines = f.readlines()

            for k in entry.keys():
                for l in lines:
                    if l.startswith(k):
                        value = l.split(':')[-1].strip()
                        if value != 'None':
                            entry[k] = float(value)
    
    return entry

In [54]:
SCENE_NAME="lego_re"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])
print(SCENE_NAME)
df

lego_re


,Avg cf,F1,psnr,name
0,0.007261,0.794973,NaN,NeRF (sig=10)
1,0.005284,0.848552,NaN,NeRF (sig=50)
2,0.007030,0.794988,NaN,NeRF (sig=100)
3,0.015506,0.593819,NaN,NeRF (sig=200)
4,0.024955,0.648561,NaN,NeRF (test mean 0.1)
5,0.004595,0.885192,NaN,NeRF (test mode 0.1)
6,0.005138,0.867795,21.381180,Ours (mlv_trunc_conv_2_28/surf_closed)
7,0.003578,0.919934,21.584274,Ours (lv_nrm_mlv_50/surf_masked)
8,0.005871,0.801840,NaN,NeuS
9,0.013204,0.742463,NaN,HFS


In [55]:
SCENE_NAME="ship_re"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])
print(SCENE_NAME)
df

ship_re


,Avg cf,F1,psnr,name
0,0.009845,0.708048,NaN,NeRF (sig=10)
1,0.006723,0.793033,NaN,NeRF (sig=50)
2,0.004787,0.874631,NaN,NeRF (sig=100)
3,0.007803,0.760769,NaN,NeRF (sig=200)
4,0.119309,0.576884,NaN,NeRF (test mean 0.1)
5,0.003026,0.948531,NaN,NeRF (test mode 0.1)
6,0.003356,0.943071,27.262106,Ours (mlv_trunc_conv_2_28/surf_closed)
7,0.002771,0.958486,27.355451,Ours (lv_nrm_mlv_50/surf_masked)
8,0.004515,0.920120,NaN,NeuS
9,0.005197,0.898159,NaN,HFS


In [56]:
SCENE_NAME="ficus"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.011315,0.806946,NaN,NeRF (sig=10)
1,0.007928,0.864619,NaN,NeRF (sig=50)
2,0.004173,0.917228,NaN,NeRF (sig=100)
3,0.012517,0.811566,NaN,NeRF (sig=200)
4,0.169846,0.320087,NaN,NeRF (test mean 0.1)
5,0.002674,0.968628,NaN,NeRF (test mode 0.1)
6,0.003735,0.967741,27.593747,Ours (mlv_trunc_conv_2_28/surf_closed)
7,0.002241,0.980546,27.256689,Ours (lv_nrm_mlv_50/surf_masked)
8,0.007811,0.818629,NaN,NeuS
9,0.009505,0.777243,NaN,HFS


In [57]:
SCENE_NAME="lyre"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.007403,0.785264,NaN,NeRF (sig=10)
1,0.008048,0.756796,NaN,NeRF (sig=50)
2,0.005425,0.823886,NaN,NeRF (sig=100)
3,0.015130,0.476366,NaN,NeRF (sig=200)
4,0.125592,0.542570,NaN,NeRF (test mean 0.1)
5,0.003033,0.961580,NaN,NeRF (test mode 0.1)
6,0.001794,0.986005,29.654643,Ours (mlv_trunc_conv_2_28/surf_closed)
7,0.001791,0.990248,29.394334,Ours (lv_nrm_mlv_50/surf_masked)
8,0.011573,0.327438,NaN,NeuS
9,0.013449,0.264055,NaN,HFS


In [58]:
SCENE_NAME="crystal"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.045442,1.872175e-01,NaN,NeRF (sig=10)
1,0.009021,5.797430e-01,NaN,NeRF (sig=50)
2,0.272388,1.683793e-03,NaN,NeRF (sig=100)
3,NaN,NaN,NaN,NeRF (sig=200)
4,0.049004,2.007382e-01,NaN,NeRF (test mean 0.1)
5,0.002947,9.748393e-01,NaN,NeRF (test mode 0.1)
6,0.023035,6.950958e-01,36.776994,Ours (mlv_trunc_conv_2_28/surf_closed)
7,0.001551,9.999297e-01,35.956169,Ours (lv_nrm_mlv_50/surf_masked)
8,0.944821,1.000000e-10,NaN,NeuS
9,0.012916,6.835514e-02,NaN,HFS


In [59]:
SCENE_NAME="materials"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.009455,0.710838,NaN,NeRF (sig=10)
1,0.009469,0.723273,NaN,NeRF (sig=50)
2,0.010249,0.725351,NaN,NeRF (sig=100)
3,0.021920,0.436452,NaN,NeRF (sig=200)
4,0.033519,0.562793,NaN,NeRF (test mean 0.1)
5,0.004118,0.902243,NaN,NeRF (test mode 0.1)
6,0.003895,0.914065,22.057884,Ours (mlv_trunc_conv_2_28/surf_closed)
7,0.003094,0.939722,21.961556,Ours (lv_nrm_mlv_50/surf_masked)
8,0.002583,0.990688,NaN,NeuS
9,0.003670,0.933831,NaN,HFS


In [60]:
SCENE_NAME="lotus"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.009859,0.689937,None,NeRF (sig=10)
1,0.005992,0.809394,None,NeRF (sig=50)
2,0.006077,0.817451,None,NeRF (sig=100)
3,0.015908,0.529068,None,NeRF (sig=200)
4,0.186448,0.363587,None,NeRF (test mean 0.1)
5,0.005688,0.854040,None,NeRF (test mode 0.1)
6,NaN,NaN,None,Ours (mlv_trunc_conv_2_28/surf_closed)
7,NaN,NaN,None,Ours (lv_nrm_mlv_50/surf_masked)
8,0.983315,0.021370,None,NeuS
9,NaN,NaN,None,HFS


In [61]:
SCENE_NAME="vase"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.022048,0.458595,None,NeRF (sig=10)
1,0.020287,0.465120,None,NeRF (sig=50)
2,0.019202,0.424388,None,NeRF (sig=100)
3,0.032311,0.207462,None,NeRF (sig=200)
4,0.045755,0.400303,None,NeRF (test mean 0.1)
5,0.011284,0.672892,None,NeRF (test mode 0.1)
6,NaN,NaN,None,Ours (mlv_trunc_conv_2_28/surf_closed)
7,NaN,NaN,None,Ours (lv_nrm_mlv_50/surf_masked)
8,0.948389,0.014227,None,NeuS
9,NaN,NaN,None,HFS


In [62]:
SCENE_NAME="maple"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.003255,0.969001,None,NeRF (sig=10)
1,0.002858,0.969782,None,NeRF (sig=50)
2,0.002874,0.954422,None,NeRF (sig=100)
3,0.004380,0.843582,None,NeRF (sig=200)
4,0.275285,0.250878,None,NeRF (test mean 0.1)
5,0.002395,0.970020,None,NeRF (test mode 0.1)
6,NaN,NaN,None,Ours (mlv_trunc_conv_2_28/surf_closed)
7,NaN,NaN,None,Ours (lv_nrm_mlv_50/surf_masked)
8,NaN,NaN,None,NeuS
9,NaN,NaN,None,HFS


In [63]:
SCENE_NAME="bulb"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.032588,0.504458,None,NeRF (sig=10)
1,0.029450,0.385860,None,NeRF (sig=50)
2,0.085343,0.103413,None,NeRF (sig=100)
3,0.189263,0.070765,None,NeRF (sig=200)
4,0.087339,0.244032,None,NeRF (test mean 0.1)
5,0.010478,0.735583,None,NeRF (test mode 0.1)
6,NaN,NaN,None,Ours (mlv_trunc_conv_2_28/surf_closed)
7,NaN,NaN,None,Ours (lv_nrm_mlv_50/surf_masked)
8,0.927899,0.004659,None,NeuS
9,NaN,NaN,None,HFS


In [64]:
SCENE_NAME="ficus_transparent"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.011620,0.826133,None,NeRF (sig=10)
1,0.007620,0.879853,None,NeRF (sig=50)
2,0.004765,0.890776,None,NeRF (sig=100)
3,0.038490,0.300974,None,NeRF (sig=200)
4,0.312655,0.221047,None,NeRF (test mean 0.1)
5,0.003056,0.974462,None,NeRF (test mode 0.1)
6,NaN,NaN,None,Ours (mlv_trunc_conv_2_28/surf_closed)
7,NaN,NaN,None,Ours (lv_nrm_mlv_50/surf_masked)
8,0.008969,0.789666,None,NeuS
9,NaN,NaN,None,HFS


In [65]:
SCENE_NAME="glasses"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,Avg cf,F1,psnr,name
0,0.053358,0.317879,None,NeRF (sig=10)
1,0.054878,0.349809,None,NeRF (sig=50)
2,0.062029,0.347310,None,NeRF (sig=100)
3,0.085853,0.200154,None,NeRF (sig=200)
4,0.453296,0.137783,None,NeRF (test mean 0.1)
5,0.051776,0.332849,None,NeRF (test mode 0.1)
6,NaN,NaN,None,Ours (mlv_trunc_conv_2_28/surf_closed)
7,NaN,NaN,None,Ours (lv_nrm_mlv_50/surf_masked)
8,0.765459,0.003165,None,NeuS
9,NaN,NaN,None,HFS
